Die lineare Diskriminantenfunktion: g(x) = w⊤x + w0 (w: Gewichtsvektor, w0: Bias oder Schwellwert) entpricht die Entscheidungsgrenze von zwei Klassen. Die Gleichung g(x) = 0 definiert eine Entscheidungsfläche, die Punkte von ω1 und ω2 trennt, damit haben wir die Entscheidungsregel: entscheide für Klasse ω1 für g(x) > 0, für ω2 für g(x) < 0, keine Entscheidung für g(x) = 0.

Unsere Aufgabe ist, die Entscheidungsgrenze (Entscheidungsfläche, Hyperebene) zu finden, welche mit einem entsprechenden Gewichtvektor w und zwei Punkten x1, x2 gilt: w⊤(x1 −x2)=0.

Wir bearbeiten jetzt mit dem Ziffer Datensatz "Digits", Quelle: https://web.stanford.edu/~hastie/ElemStatLearn/
Dann Implementieren wir eine binäre Klassifikator, die zwei Klassen voneinander unterscheiden können mittels linearer Regression. Ein Klassifikator sollte also beispielsweise 3en von 5en unterscheiden können.

In [42]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from scipy import linalg
from numpy.linalg import inv, LinAlgError, cond
import math

In [43]:
def file_get_contents(filename):
    return np.genfromtxt(filename, delimiter=',')

Wir vorbereiten zwei Ziffernmenge, welche die Ziffern 3 und Ziffern 5 darstellt.

In [44]:
t3 = file_get_contents('train.3')
t5 = file_get_contents('train.5')

Zur Vereinfachung der Berechnung, fügen wir eine Spalte mit 1'en bei den Trainingdaten hinzu, so dass es keine w0 mit Sampeldata multiplizieren muss, damit wir statt w und w0, nur Gewichtvektor w betrachten müssen.

In [45]:
def konfiguration(matrix): 
    length = len(matrix)
    column = np.ones((length,1))
    return np.concatenate((column,matrix),1)

Um eine Inversematrix von Gewichtvektor (Matrix) zu berechnen, müssen wir zuerst garantieren, es überhaupt invertierbar ist.

In [46]:
def get_it_invertible(matrix):
    a = 0.1
    id = np.identity(len(matrix))
    deter = np.linalg.det(matrix)

    while (deter == 0) :  # wenn Determinante gleich 0, gibt's keine Inverse Matrix
        matrix = np.add(matrix, np.dot(a, id))  # dann addieren wir mit eine skalarierte Einheitsmatrix
        deter = np.linalg.det(matrix)
    return matrix

In [47]:
def setvector(matrix, length):  # ein Vektor mit wechselseitige -1 und 1, unsere 'y Vektor'
    vector = np.array(np.ones((length,1)))
    for i in range (0, length):
        if (math.fmod(i,2) == 1):
            vector[i] = -1
    return vector

Jetzt berechnen wir den Gewichtvektor, mittels lineare Diskriminante.

In [48]:
def get_fit(matrix_minus): 
    matrix = konfiguration(matrix_minus)
    transport = np.transpose(matrix)
    vektor = np.dot(transport, matrix)

    inverse_vektor = np.linalg.inv(get_it_invertible(vektor))
    tmp = np.dot(inverse_vektor, transport)
    tmp_set = setvector(tmp, len(tmp[1]))
    return np.dot(tmp, tmp_set)

Damit können wir schon Voraussage geben, welches Bild zu welche Ziffern gehören.

In [49]:
def prediction(matrix1, matrix2, test):
    beta = np.matrix(get_fit(matrix1[0]))
    test_vector = np.matrix(np.insert(test,0,0))  # Test Vektor hat eine Stelle weniger wegen Konfiguration

    matrix = np.dot(test_vector, beta)
    mysum = np.sum(matrix)
    if (mysum > 0):  
        return matrix2[1]
    else:
        return matrix1[1]

In [50]:
def get_accuracy(matrix1, matrix2, testmatrix):  
    error = 0
    expect = testmatrix[1]
    length = len(testmatrix[0])

    for i in range(0, len(testmatrix[0])):
        result = prediction(matrix1, matrix2, testmatrix[0][i])
        if (result != expect):
            error = error + 1  
    return (error / length)

Jetzt schauen wir uns mal das Ergebnis an.

In [51]:
print(get_accuracy([t5, 5], [t3, 3], [t5, 5]))

/Users/hongzhu/anaconda3/lib/python3.6/site-packages/numpy/linalg/linalg.py:1874: RuntimeWarning: overflow encountered in det
  r = _umath_linalg.det(a, signature=signature)


0.0
